Import necessary packages, define API key, and connect to database in order to pull list of 5,000 most populous cities in the US.

In [1]:
import requests
import config
import json

In [2]:
api_key = config.api_key


In [3]:
db_name = 'City_Energy_Data'
import mysql.connector
import config

In [4]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [5]:
cur.execute("""SELECT *
                      FROM population_data
                    ;""")
pop_data = cur.fetchall()
pop_data[0:5]

[(1, 'New York ', 'NY', '8,363,710'),
 (2, 'Los Angeles ', 'CA', '3,833,995'),
 (3, 'Chicago ', 'IL', '2,853,114'),
 (4, 'Houston ', 'TX', '2,242,193'),
 (5, 'Phoenix ', 'AZ', '1,567,924')]

Loop through list to isolate city name and city state.

In [6]:
city_names = []
for city in pop_data:
    name = city[1]
    name = name.strip()
    city_names.append(name)


In [8]:
city_states = []
for city in pop_data:
    state = city[2]
    state = state.strip()
    city_states.append(state)
len(city_states)

5000

Create tables for residential, commercial, and industrial energy data with city_id as primary key.

In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [ ]:
tables_res = {}
tables_res['residential_energy_A'] = (
    """CREATE TABLE residential_energy_A (
       housing_units int(40) ,
       elec_mwh int(40),
       gas_mcf int(40),
       elec_lb_ghg int(40),
       gas_lb_ghg int(40),
       city varchar(40),
       state varchar(40),
       city_id int(40),
       PRIMARY KEY (city_id))"""
) 


In [ ]:
tables_com = {}
tables_com['commercial_energy_A'] = (
    """CREATE TABLE commercial_energy_A (
       num_establishments int(40) ,
       elec_mwh int(40),
       gas_mcf int(40),
       elec_lb_ghg int(40),
       gas_lb_ghg int(40),
       city varchar(40),
       state varchar(40),
       city_id int(40),
       PRIMARY KEY (city_id))"""
) 

In [ ]:
tables_ind = {}
tables_ind['industrial_energy_A'] = (
    """CREATE TABLE industrial_energy_A (
       num_establishments int(40) ,
       elec_mwh int(40),
       gas_mcf int(40),
       elec_lb_ghg int(40),
       gas_lb_ghg int(40),
       city varchar(40),
       state varchar(40),
       city_id int(40),
       PRIMARY KEY (city_id))"""
) 

In [ ]:
tables_res['residential_energy_A']

In [ ]:
cur.execute(tables_res['residential_energy_A'])
cnx.commit()

In [ ]:
cur.execute(tables_com['commercial_energy_A'])
cnx.commit()

In [ ]:
cur.execute(tables_ind['industrial_energy_A'])
cnx.commit()

In [ ]:
cnx.close()

Loop list of cities and states through get_energy_info function to obtain residential, commercial, and industrial energy data for each city. Given the 1,000 API calls per hour limit, loop through cities as groups of 500. 

In [1]:
energy_data_1 = []
for x, y in zip(city_names[0:50], city_states[0:50]):
    output = get_energy_info(x, y)
    energy_data_1.append(output)
    


Check results. City energy data should be a list of tuples, with each tuple containing three dictionaries. 

In [16]:
energy_data_1

[({'housing_units': 3128246,
   'elec_mwh': 14872960,
   'gas_mcf': 179425068,
   'elec_lb_ghg': 22285615334,
   'gas_lb_ghg': 21538668054,
   'city': 'New York',
   'state': 'NY'},
  {'num_establishments': 92276,
   'elec_mwh': 26024809,
   'gas_mcf': 97518263,
   'elec_lb_ghg': 38995524672,
   'gas_lb_ghg': 11706354721,
   'city': 'New York',
   'state': 'NY'},
  {'num_establishments': 26376,
   'elec_mwh': 1245512,
   'gas_mcf': 5426336,
   'elec_lb_ghg': 1866272757,
   'gas_lb_ghg': 651391971,
   'city': 'New York',
   'state': 'NY'}),
 ({'housing_units': 1355074,
   'elec_mwh': 9043891,
   'gas_mcf': 41818846,
   'elec_lb_ghg': 12109857609,
   'gas_lb_ghg': 5020046832,
   'city': 'Los Angeles',
   'state': 'CA'},
  {'num_establishments': 49649,
   'elec_mwh': 12023369,
   'gas_mcf': 14889777,
   'elec_lb_ghg': 16099407399,
   'gas_lb_ghg': 1787408917,
   'city': 'Los Angeles',
   'state': 'CA'},
  {'num_establishments': 24184,
   'elec_mwh': 2288471,
   'gas_mcf': 34767844,
   'el

Prepare residential data for the database. This includes selecting residential sector information from each city's  tuple and providing each city with a corresponding city_id (population rank).

In [17]:
list_of_residential_1 = []
for i in energy_data_1:
    res_info = i[0]
    list_of_residential_1.append(res_info)

In [18]:
for index, item in enumerate(list_of_residential_1):
    list_of_residential_1[index].update( {'city_id' : index+4501} )
    

Check residential data to make sure it has been modified correctly. 

In [30]:
list_of_residential_1[5]

{'housing_units': 582511,
 'elec_mwh': 4271528,
 'gas_mcf': 33972181,
 'elec_lb_ghg': 8231817377,
 'gas_lb_ghg': 4078112104,
 'city': 'Philadelphia',
 'state': 'PA',
 'city_id': 4506}

In [25]:
list_of_residential_1[-1] #id should match city_names index + 1 

{'housing_units': 134895,
 'elec_mwh': 1923102,
 'gas_mcf': 2071627,
 'elec_lb_ghg': 2575036519,
 'gas_lb_ghg': 248683627,
 'city': 'Arlington',
 'state': 'TX',
 'city_id': 4550}

Loop through residential data to check for missing values and return clean list of dictionaries. 

In [26]:
list_of_residential_1 = [item for item in list_of_residential_1 if item['elec_mwh'] != 'no_info']

Check the amount of missing values.

In [27]:
len(list_of_residential_1)

47

In [31]:
list_of_residential_1[5] #id should match city_names index + 1 

{'housing_units': 582511,
 'elec_mwh': 4271528,
 'gas_mcf': 33972181,
 'elec_lb_ghg': 8231817377,
 'gas_lb_ghg': 4078112104,
 'city': 'Philadelphia',
 'state': 'PA',
 'city_id': 4506}

Follow same directions in order to prepare commercial data.

In [889]:
list_of_commercial_1 = []
for i in energy_data_1:
    com_info = i[1]
    list_of_commercial_1.append(com_info)

In [891]:
for index, item in enumerate(list_of_commercial_1):
    list_of_commercial_1[index].update( {'city_id' : index+4501} )

In [893]:
list_of_commercial_1 = [item for item in list_of_commercial_1 if item['elec_mwh'] != 'no_info']

In [894]:
list_of_commercial_1[0]

{'num_establishments': 68,
 'elec_mwh': 13653,
 'gas_mcf': 33079,
 'elec_lb_ghg': 17820142,
 'gas_lb_ghg': 3970932,
 'city': 'New Haven',
 'state': 'MI',
 'city_id': 4501}

In [895]:
len(list_of_commercial_1)

482

Follow same directions in order to prepare industrial data.

In [896]:
list_of_industrial_1 = []
for i in energy_data_1:
    ind_info = i[2]
    list_of_industrial_1.append(ind_info)

In [897]:
for index, item in enumerate(list_of_industrial_1):
    list_of_industrial_1[index].update( {'city_id' : index+4501} )

In [899]:
list_of_industrial_1 = [item for item in list_of_industrial_1 if item['elec_mwh'] != 'no_info']

In [900]:
len(list_of_industrial_1)

482

In [901]:
list_of_industrial_1[-1]

{'num_establishments': 9,
 'elec_mwh': 4178,
 'gas_mcf': 16126,
 'elec_lb_ghg': 5180018,
 'gas_lb_ghg': 1935845,
 'city': 'Newport',
 'state': 'NC',
 'city_id': 5000}

Insert residential data dictionaries into the residential_energy_A table. 

In [902]:
add_residential = ("INSERT INTO residential_energy_A "
               "(housing_units, elec_mwh, gas_mcf, elec_lb_ghg, gas_lb_ghg, city, state, city_id) "
               "VALUES (%(housing_units)s, %(elec_mwh)s, %(gas_mcf)s, %(elec_lb_ghg)s, %(gas_lb_ghg)s, %(city)s, %(state)s, %(city_id)s)")



In [903]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [904]:
cur.executemany(add_residential, list_of_residential_1)
cnx.commit()

In [905]:
cur.close()
cnx.close()

Insert commercial data into commercial_energy_A table. 

In [906]:
add_commercial = ("INSERT INTO commercial_energy_A "
               "(num_establishments, elec_mwh, gas_mcf, elec_lb_ghg, gas_lb_ghg, city, state, city_id) "
               "VALUES (%(num_establishments)s, %(elec_mwh)s, %(gas_mcf)s, %(elec_lb_ghg)s, %(gas_lb_ghg)s, %(city)s, %(state)s, %(city_id)s)")



In [907]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [908]:
cur.executemany(add_commercial, list_of_commercial_1)
cnx.commit()

Insert industrial data into industrial_energy_A table.

In [909]:
add_industrial = ("INSERT INTO industrial_energy_A "
               "(num_establishments, elec_mwh, gas_mcf, elec_lb_ghg, gas_lb_ghg, city, state, city_id) "
               "VALUES (%(num_establishments)s, %(elec_mwh)s, %(gas_mcf)s, %(elec_lb_ghg)s, %(gas_lb_ghg)s, %(city)s, %(state)s, %(city_id)s)")

In [910]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [911]:
cur.executemany(add_industrial, list_of_industrial_1)
cnx.commit()

In [912]:
cnx.close()